# trainデータの作成

trainデータは以下のようにする
- 馬ごとの過去3回のレースデータを保持
- 過去のデータが3回未満である場合は0で穴埋め
- ped_id で紐付けされた血統の情報をもとに 祖先の産駒成績を merge
- target はその馬が、そのレースにおいて何着になったかを3つに分類（着順上位(1~3着)/中位(4~7着)/下位(8~)の予測）

In [93]:
import numpy as np
import pandas as pd
#最大表示列数の指定（ここでは300列を指定）
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)

In [9]:
race_df = pd.read_csv("./csv_cleaned/race_data.csv", sep=",")
horse_df = pd.read_csv("./csv_cleaned/horse_data.csv", sep=",")
sire_df = pd.read_csv("./csv_cleaned/sire.csv", sep=",")
ped_id_df = pd.read_csv("./csv_cleaned/ped_id.csv", sep=",")

In [14]:
# race
race_df.tail(3)

,race_id,race_round,ground_status,date,total_horse_number,frame_number_first,horse_number_first,frame_number_second,horse_number_second,frame_number_third,horse_number_third,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wakuren,umaren,wide_1_2,wide_1_3,wide_2_3,umatan,renhuku3,rentan3,distance,weather_rain,weather_snow
3039,201905050608,8,1,2019-11-17 13:45:00,16,2,3,1,2,5,10,380,160,420,170,3090,4130,1300,370,1730,6620,6020,34340,1400,0,0
3040,201905021209,9,1,2019-05-26 14:15:00,15,6,10,3,4,8,15,420,190,270,140,850,2140,890,380,690,3390,3040,14990,1600,0,0
3041,201905010104,4,1,2019-01-26 11:40:00,14,8,13,3,4,6,10,340,170,170,480,550,760,380,1130,1600,1480,3870,16710,1600,0,0


In [15]:
# horse
horse_df['date'] = pd.to_datetime(horse_df['date'])
horse_df.head(3)

,race_id,rank,frame_number,horse_number,horse_id,age,burden_weight,rider_id,goal_time,half_way_rank,last_time,odds,popular,horse_weight,tamer_id,owner_id,date,is_down,is_senba,is_mesu,is_osu,distance,avg_velocity,horse_weight_dif,burden_weight_rate
0,200805010502,1,3,4,2005105185,3,54.0,1085,80.6,1.0,37.7,7.6,3.0,454.0,1017,853002,2008-02-16 10:30:00,0,0,1,0,1300,16.129032,0.0,0.118943
1,200805010502,2,6,10,2005106076,3,54.0,663,80.6,2.5,37.6,5.1,2.0,478.0,1064,138800,2008-02-16 10:30:00,0,0,1,0,1300,16.129032,0.0,0.112971
2,200805010502,3,8,14,2005102725,3,56.0,684,81.2,7.5,37.6,10.2,4.0,474.0,1074,226800,2008-02-16 10:30:00,0,0,0,1,1300,16.009852,0.0,0.118143


In [16]:
# sire
sire_df.head(3)

,ped_id,rank,win_horse_rate,win_times_rate,win_jusyo_rate,win_special_rate,win_plane_rate,win_turf_rate,win_dirt_rate,EI
0,000a00033b,571.0,0.250000,0.049151,0.000000,0.048889,0.050114,0.060000,0.032120,0.51
1,000a012187,2084.0,1.000000,0.104167,0.000000,0.000000,0.138889,0.000000,0.116279,1.30
2,000a001d37,404.0,0.568627,0.114537,0.104167,0.112033,0.117347,0.120944,0.110429,1.90


In [17]:
# ped_id
ped_id_df.head(3)

,horse_id,f_id,mf_id,mmf_id,fmf_id
0,2015100929,2001103890,000a0022a2,000a001a8f,000a001d7e
1,2007105538,1995103211,1989109102,000a0003bb,000a0003bd
2,2015100933,2001100650,000a000d69,000a000249,000a001a98


## raceからの追加

In [35]:
# raceからのmerge
# 'distance'はすでにある
race_df_for_merge = race_df[['race_id','ground_status','total_horse_number','weather_rain','weather_snow']]

merged_horse_df = pd.merge(horse_df, race_df_for_merge, on='race_id')

In [36]:
merged_horse_df.head(2)

,race_id,rank,frame_number,horse_number,horse_id,age,burden_weight,rider_id,goal_time,half_way_rank,last_time,odds,popular,horse_weight,tamer_id,owner_id,date,is_down,is_senba,is_mesu,is_osu,distance,avg_velocity,horse_weight_dif,burden_weight_rate,ground_status,total_horse_number,weather_rain,weather_snow
0,200805010502,1,3,4,2005105185,3,54.0,1085,80.6,1.0,37.7,7.6,3.0,454.0,1017,853002,2008-02-16 10:30:00,0,0,1,0,1300,16.129032,0.0,0.118943,4,15,0,0
1,200805010502,2,6,10,2005106076,3,54.0,663,80.6,2.5,37.6,5.1,2.0,478.0,1064,138800,2008-02-16 10:30:00,0,0,1,0,1300,16.129032,0.0,0.112971,4,15,0,0


In [37]:
merged_horse_df.shape

(46150, 29)

## 標準化・正規化しやすいように先にfloatへ

In [38]:
merged_horse_df[merged_horse_df.columns[(merged_horse_df.columns != 'race_id')&(merged_horse_df.columns != 'horse_id')&(merged_horse_df.columns != 'date')&(merged_horse_df.columns != 'owner_id')&(merged_horse_df.columns != 'tamer_id')&(merged_horse_df.columns != 'rider_id')]] = merged_horse_df[merged_horse_df.columns[(merged_horse_df.columns != 'race_id')&(merged_horse_df.columns != 'horse_id')&(merged_horse_df.columns != 'date')&(merged_horse_df.columns != 'owner_id')&(merged_horse_df.columns != 'tamer_id')&(merged_horse_df.columns != 'rider_id')]].astype(float)

In [39]:
print(merged_horse_df.dtypes)
merged_horse_df.shape

race_id                        int64
rank                         float64
frame_number                 float64
horse_number                 float64
horse_id                       int64
age                          float64
burden_weight                float64
rider_id                       int64
goal_time                    float64
half_way_rank                float64
last_time                    float64
odds                         float64
popular                      float64
horse_weight                 float64
tamer_id                       int64
owner_id                       int64
date                  datetime64[ns]
is_down                      float64
is_senba                     float64
is_mesu                      float64
is_osu                       float64
distance                     float64
avg_velocity                 float64
horse_weight_dif             float64
burden_weight_rate           float64
ground_status                float64
total_horse_number           float64
w

(46150, 29)

## 季節情報としてsin,cosを入れる
ありだけど今回はとりあえず見送る

## 馬の前回までのレース情報をshiftして結合
group by してからそれぞれ計算するので、時間がかかる

追加項目
- 前回のレース日時からの差
- rider_idの変化があるか(あるなら1)

In [40]:
# group by したデータフレームに対して行う処理
def make_one_horse_train_data(one_horse_data):
    one_horse_data = one_horse_data.sort_values('date',ascending=False)
    one_horse_data['pre_date_diff'] = one_horse_data['date'].diff(-1).dt.days
    one_horse_data['is_rider_same'] = (one_horse_data['rider_id'].shift(-1) == one_horse_data['rider_id']) * 1.0
    one_horse_data['is_tamer_same'] = (one_horse_data['tamer_id'].shift(-1) == one_horse_data['tamer_id']) * 1.0
    one_horse_data['is_owner_same'] = (one_horse_data['owner_id'].shift(-1) == one_horse_data['owner_id']) * 1.0

    #不要なので削除
    one_horse_data.drop(['rider_id'], axis=1, inplace=True)
    one_horse_data.drop(['tamer_id'], axis=1, inplace=True)
    one_horse_data.drop(['owner_id'], axis=1, inplace=True)


    #  数レース分の情報を結合
    result_df = one_horse_data.copy()
    
    for i in range(1,4):
        # i だけ shiftしたものを得る
        # race_id, sexはいらない
        shift_df = one_horse_data.drop(['race_id','is_senba','is_mesu','is_osu'], axis=1).shift(-i)
        # 目的のレースとどれだけの時間離れているか？
        shift_df['interval_date'] = (one_horse_data['date'] - shift_df['date']).dt.days
        shift_df.drop(['date'], axis=1, inplace=True)
        shift_df.columns = shift_df.columns + "_" + str(i)
        result_df = pd.concat([result_df, shift_df], axis=1)
    return result_df



In [41]:
# test
one_horse_data = merged_horse_df[merged_horse_df['horse_id'] == 2004110053].copy()
make_one_horse_train_data(one_horse_data).head()

,race_id,rank,frame_number,horse_number,horse_id,age,burden_weight,goal_time,half_way_rank,last_time,odds,popular,horse_weight,date,is_down,is_senba,is_mesu,is_osu,distance,avg_velocity,horse_weight_dif,burden_weight_rate,ground_status,total_horse_number,weather_rain,weather_snow,pre_date_diff,is_rider_same,is_tamer_same,is_owner_same,rank_1,frame_number_1,horse_number_1,horse_id_1,age_1,burden_weight_1,goal_time_1,half_way_rank_1,last_time_1,odds_1,popular_1,horse_weight_1,is_down_1,distance_1,avg_velocity_1,horse_weight_dif_1,burden_weight_rate_1,ground_status_1,total_horse_number_1,weather_rain_1,weather_snow_1,pre_date_diff_1,is_rider_same_1,is_tamer_same_1,is_owner_same_1,interval_date_1,rank_2,frame_number_2,horse_number_2,horse_id_2,age_2,burden_weight_2,goal_time_2,half_way_rank_2,last_time_2,odds_2,popular_2,horse_weight_2,is_down_2,distance_2,avg_velocity_2,horse_weight_dif_2,burden_weight_rate_2,ground_status_2,total_horse_number_2,weather_rain_2,weather_snow_2,pre_date_diff_2,is_rider_same_2,is_tamer_same_2,is_owner_same_2,interval_date_2,rank_3,frame_number_3,horse_number_3,horse_id_3,age_3,burden_weight_3,goal_time_3,half_way_rank_3,last_time_3,odds_3,popular_3,horse_weight_3,is_down_3,distance_3,avg_velocity_3,horse_weight_dif_3,burden_weight_rate_3,ground_status_3,total_horse_number_3,weather_rain_3,weather_snow_3,pre_date_diff_3,is_rider_same_3,is_tamer_same_3,is_owner_same_3,interval_date_3
355,200805040812,5.0,2.0,4.0,2004110053,4.0,57.0,98.1,4.0,36.4,8.6,5.0,490.0,2008-11-02 16:30:00,0.0,0.0,0.0,1.0,1600.0,16.309888,0.0,0.116327,1.0,16.0,0.0,0.0,7.0,1.0,1.0,1.0,1.0,6.0,10.0,2.004110e+09,4.0,57.0,97.2,2.0,36.4,16.9,6.0,490.0,0.0,1600.0,16.460905,-8.0,0.116327,4.0,15.0,0.0,0.0,NaN,0.0,0.0,0.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2809,200805040607,1.0,6.0,10.0,2004110053,4.0,57.0,97.2,2.0,36.4,16.9,6.0,490.0,2008-10-26 13:10:00,0.0,0.0,0.0,1.0,1600.0,16.460905,-8.0,0.116327,4.0,15.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
race_and_score_df = merged_horse_df.sort_values('date',ascending=False).groupby('horse_id', as_index=False).apply(make_one_horse_train_data)


In [43]:
race_and_score_df.head(2)

race_id  rank  frame_number  horse_number    horse_id   age  \
0 887   200805010811  14.0           1.0           2.0  1996110169  12.0   
  1404  200805010211   7.0           6.0          12.0  1996110169  12.0   

        burden_weight  goal_time  half_way_rank  last_time   odds  popular  \
0 887            57.0       97.9           10.0       37.9  165.7     15.0   
  1404           58.0       83.4           13.5       36.1  172.1     16.0   

        horse_weight                date  is_down  is_senba  is_mesu  is_osu  \
0 887          456.0 2008-02-24 15:40:00      0.0       0.0      0.0     1.0   
  1404         456.0 2008-02-04 15:40:00      0.0       0.0      0.0     1.0   

        distance  avg_velocity  horse_weight_dif  burden_weight_rate  \
0 887     1600.0     16.343207               0.0            0.125000   
  1404    1400.0     16.786571              -2.0            0.127193   

        ground_status  total_horse_number  weather_rain  weather_snow  \
0 887             1.0                16.0           0.0           0.0   
  1404            2.0                16.0           0.0           0.0   

        pre_date_diff  is_rider_same  is_tamer_same  is_owner_same  rank_1  \
0 887            20.0            0.0            1.0            1.0     7.0   
  1404            NaN            0.0            0.0            0.0     NaN   

        frame_number_1  horse_number_1    horse_id_1  age_1  burden_weight_1  \
0 887              6.0            12.0  1.996110e+09   12.0             58.0   
  1404             NaN             NaN           NaN    NaN              NaN   

        goal_time_1  half_way_rank_1  last_time_1  odds_1  popular_1  \
0 887          83.4             13.5         36.1   172.1       16.0   
  1404          NaN              NaN          NaN     NaN        NaN   

        horse_weight_1  is_down_1  distance_1  avg_velocity_1  \
0 887            456.0        0.0      1400.0       16.786571   
  1404             NaN        NaN         NaN             NaN   

        horse_weight_dif_1  burden_weight_rate_1  ground_status_1  \
0 887                 -2.0              0.127193              2.0   
  1404                 NaN                   NaN              NaN   

        total_horse_number_1  weather_rain_1  weather_snow_1  pre_date_diff_1  \
0 887                   16.0             0.0             0.0              NaN   
  1404                   NaN             NaN             NaN              NaN   

        is_rider_same_1  is_tamer_same_1  is_owner_same_1  interval_date_1  \
0 887               0.0              0.0              0.0             20.0   
  1404              NaN              NaN              NaN              NaN   

        rank_2  frame_number_2  horse_number_2  horse_id_2  age_2  \
0 887      NaN             NaN             NaN         NaN    NaN   
  1404     NaN             NaN             NaN         NaN    NaN   

        burden_weight_2  goal_time_2  half_way_rank_2  last_time_2  odds_2  \
0 887               NaN          NaN              NaN          NaN     NaN   
  1404              NaN          NaN              NaN          NaN     NaN   

        popular_2  horse_weight_2  is_down_2  distance_2  avg_velocity_2  \
0 887         NaN             NaN        NaN         NaN             NaN   
  1404        NaN             NaN        NaN         NaN             NaN   

        horse_weight_dif_2  burden_weight_rate_2  ground_status_2  \
0 887                  NaN                   NaN              NaN   
  1404                 NaN                   NaN              NaN   

        total_horse_number_2  weather_rain_2  weather_snow_2  pre_date_diff_2  \
0 887                    NaN             NaN             NaN              NaN   
  1404                   NaN             NaN             NaN              NaN   

        is_rider_same_2  is_tamer_same_2  is_owner_same_2  interval_date_2  \
0 887               NaN              NaN              NaN              NaN   
  1404         

In [44]:
# 時間がかかったので結果を保存しておく
race_and_score_df.to_csv("csv_merged/race_and_score.csv", index=False )

## 血統情報の追加
ped_id_csv をもとに対応する祖先のidを取得

sire_csv から祖先に対応する成績を merge


In [96]:
df = race_and_score_df

In [97]:
# 祖先のidをmergeで取得
df = pd.merge(ped_id_df, df, on='horse_id')
df.head(3)

,horse_id,f_id,mf_id,mmf_id,fmf_id,race_id,rank,frame_number,horse_number,age,burden_weight,goal_time,half_way_rank,last_time,odds,popular,horse_weight,date,is_down,is_senba,is_mesu,is_osu,distance,avg_velocity,horse_weight_dif,burden_weight_rate,ground_status,total_horse_number,weather_rain,weather_snow,pre_date_diff,is_rider_same,is_tamer_same,is_owner_same,rank_1,frame_number_1,horse_number_1,horse_id_1,age_1,burden_weight_1,goal_time_1,half_way_rank_1,last_time_1,odds_1,popular_1,horse_weight_1,is_down_1,distance_1,avg_velocity_1,horse_weight_dif_1,burden_weight_rate_1,ground_status_1,total_horse_number_1,weather_rain_1,weather_snow_1,pre_date_diff_1,is_rider_same_1,is_tamer_same_1,is_owner_same_1,interval_date_1,rank_2,frame_number_2,horse_number_2,horse_id_2,age_2,burden_weight_2,goal_time_2,half_way_rank_2,last_time_2,odds_2,popular_2,horse_weight_2,is_down_2,distance_2,avg_velocity_2,horse_weight_dif_2,burden_weight_rate_2,ground_status_2,total_horse_number_2,weather_rain_2,weather_snow_2,pre_date_diff_2,is_rider_same_2,is_tamer_same_2,is_owner_same_2,interval_date_2,rank_3,frame_number_3,horse_number_3,horse_id_3,age_3,burden_weight_3,goal_time_3,half_way_rank_3,last_time_3,odds_3,popular_3,horse_weight_3,is_down_3,distance_3,avg_velocity_3,horse_weight_dif_3,burden_weight_rate_3,ground_status_3,total_horse_number_3,weather_rain_3,weather_snow_3,pre_date_diff_3,is_rider_same_3,is_tamer_same_3,is_owner_same_3,interval_date_3
0,2015100929,2001103890,000a0022a2,000a001a8f,000a001d7e,201905030412,15.0,4.0,8.0,4.0,55.0,98.1,11.5,37.2,37.2,10.0,468.0,2019-06-09 16:25:00,0.0,0.0,1.0,0.0,1600.0,16.309888,6.0,0.117521,3.0,16.0,2.0,0.0,218.0,0.0,1.0,1.0,9.0,6.0,7.0,2.015101e+09,3.0,54.0,99.8,10.5,38.0,17.9,5.0,462.0,0.0,1600.0,16.032064,-2.0,0.116883,1.0,11.0,0.0,0.0,14.0,1.0,1.0,1.0,218.0,3.0,8.0,11.0,2.015101e+09,3.0,53.0,98.7,4.5,37.1,31.0,9.0,464.0,0.0,1600.0,16.210740,4.0,0.114224,1.0,11.0,0.0,0.0,12.0,0.0,1.0,1.0,232.0,8.0,2.0,3.0,2.015101e+09,3.0,53.0,100.1,13.0,37.2,12.9,6.0,460.0,0.0,1600.0,15.984016,-8.0,0.115217,1.0,16.0,0.0,0.0,106.0,0.0,1.0,1.0,244.0
1,2015100929,2001103890,000a0022a2,000a001a8f,000a001d7e,201805050108,9.0,6.0,7.0,3.0,54.0,99.8,10.5,38.0,17.9,5.0,462.0,2018-11-03 13:55:00,0.0,0.0,1.0,0.0,1600.0,16.032064,-2.0,0.116883,1.0,11.0,0.0,0.0,14.0,1.0,1.0,1.0,3.0,8.0,11.0,2.015101e+09,3.0,53.0,98.7,4.5,37.1,31.0,9.0,464.0,0.0,1600.0,16.210740,4.0,0.114224,1.0,11.0,0.0,0.0,12.0,0.0,1.0,1.0,14.0,8.0,2.0,3.0,2.015101e+09,3.0,53.0,100.1,13.0,37.2,12.9,6.0,460.0,0.0,1600.0,15.984016,-8.0,0.115217,1.0,16.0,0.0,0.0,106.0,0.0,1.0,1.0,26.0,1.0,2.0,2.0,2.015101e+09,3.0,54.0,97.4,11.0,36.2,7.2,3.0,464.0,0.0,1600.0,16.427105,-4.0,0.116379,2.0,15.0,1.0,0.0,13.0,0.0,1.0,1.0,132.0
2,2015100929,2001103890,000a0022a2,000a001a8f,000a001d7e,201805040607,3.0,8.0,11.0,3.0,53.0,98.7,4.5,37.1,31.0,9.0,464.0,2018-10-20 13:35:00,0.0,0.0,1.0,0.0,1600.0,16.210740,4.0,0.114224,1.0,11.0,0.0,0.0,12.0,0.0,1.0,1.0,8.0,2.0,3.0,2.015101e+09,3.0,53.0,100.1,13.0,37.2,12.9,6.0,460.0,0.0,1600.0,15.984016,-8.0,0.115217,1.0,16.0,0.0,0.0,106.0,0.0,1.0,1.0,12.0,1.0,2.0,2.0,2.015101e+09,3.0,54.0,97.4,11.0,36.2,7.2,3.0,464.0,0.0,1600.0,16.427105,-4.0,0.116379,2.0,15.0,1.0,0.0,13.0,0.0,1.0,1.0,118.0,3.0,3.0,5.0,2.015101e+09,3.0,54.0,100.2,15.0,36.8,111.9,12.0,468.0,0.0,1600.0,15.968064,6.0,0.115385,1.0,16.0,2.0,0.0,NaN,0.0,0.0,0.0,132.0


In [98]:
# 祖先ごとに成績をmerge
df = pd.merge(df, sire_df, left_on='f_id',right_on='ped_id' , suffixes=['', '_f'])
df = pd.merge(df, sire_df, left_on='mf_id',right_on='ped_id', suffixes=['', '_mf'])
df = pd.merge(df, sire_df, left_on='mmf_id',right_on='ped_id', suffixes=['', '_mmf'])
df = pd.merge(df, sire_df, left_on='fmf_id',right_on='ped_id', suffixes=['', '_fmf'])

In [99]:
# horse_id、祖先のidは不要になったので消去
df.drop(['horse_id'], axis=1, inplace=True)
df.drop(['f_id'], axis=1, inplace=True)
df.drop(['mf_id'], axis=1, inplace=True)
df.drop(['mmf_id'], axis=1, inplace=True)
df.drop(['fmf_id'], axis=1, inplace=True)
df.drop(['ped_id'], axis=1, inplace=True)
df.drop(['ped_id_mf'], axis=1, inplace=True)
df.drop(['ped_id_mmf'], axis=1, inplace=True)
df.drop(['ped_id_fmf'], axis=1, inplace=True)

df.head(3)

,race_id,rank,frame_number,horse_number,age,burden_weight,goal_time,half_way_rank,last_time,odds,popular,horse_weight,date,is_down,is_senba,is_mesu,is_osu,distance,avg_velocity,horse_weight_dif,burden_weight_rate,ground_status,total_horse_number,weather_rain,weather_snow,pre_date_diff,is_rider_same,is_tamer_same,is_owner_same,rank_1,frame_number_1,horse_number_1,horse_id_1,age_1,burden_weight_1,goal_time_1,half_way_rank_1,last_time_1,odds_1,popular_1,horse_weight_1,is_down_1,distance_1,avg_velocity_1,horse_weight_dif_1,burden_weight_rate_1,ground_status_1,total_horse_number_1,weather_rain_1,weather_snow_1,pre_date_diff_1,is_rider_same_1,is_tamer_same_1,is_owner_same_1,interval_date_1,rank_2,frame_number_2,horse_number_2,horse_id_2,age_2,burden_weight_2,goal_time_2,half_way_rank_2,last_time_2,odds_2,popular_2,horse_weight_2,is_down_2,distance_2,avg_velocity_2,horse_weight_dif_2,burden_weight_rate_2,ground_status_2,total_horse_number_2,weather_rain_2,weather_snow_2,pre_date_diff_2,is_rider_same_2,is_tamer_same_2,is_owner_same_2,interval_date_2,rank_3,frame_number_3,horse_number_3,horse_id_3,age_3,burden_weight_3,goal_time_3,half_way_rank_3,last_time_3,odds_3,popular_3,horse_weight_3,is_down_3,distance_3,avg_velocity_3,horse_weight_dif_3,burden_weight_rate_3,ground_status_3,total_horse_number_3,weather_rain_3,weather_snow_3,pre_date_diff_3,is_rider_same_3,is_tamer_same_3,is_owner_same_3,interval_date_3,rank_f,win_horse_rate,win_times_rate,win_jusyo_rate,win_special_rate,win_plane_rate,win_turf_rate,win_dirt_rate,EI,rank_mf,win_horse_rate_mf,win_times_rate_mf,win_jusyo_rate_mf,win_special_rate_mf,win_plane_rate_mf,win_turf_rate_mf,win_dirt_rate_mf,EI_mf,rank_mmf,win_horse_rate_mmf,win_times_rate_mmf,win_jusyo_rate_mmf,win_special_rate_mmf,win_plane_rate_mmf,win_turf_rate_mmf,win_dirt_rate_mmf,EI_mmf,rank_fmf,win_horse_rate_fmf,win_times_rate_fmf,win_jusyo_rate_fmf,win_special_rate_fmf,win_plane_rate_fmf,win_turf_rate_fmf,win_dirt_rate_fmf,EI_fmf
0,201905030412,15.0,4.0,8.0,4.0,55.0,98.1,11.5,37.2,37.2,10.0,468.0,2019-06-09 16:25:00,0.0,0.0,1.0,0.0,1600.0,16.309888,6.0,0.117521,3.0,16.0,2.0,0.0,218.0,0.0,1.0,1.0,9.0,6.0,7.0,2.015101e+09,3.0,54.0,99.8,10.5,38.0,17.9,5.0,462.0,0.0,1600.0,16.032064,-2.0,0.116883,1.0,11.0,0.0,0.0,14.0,1.0,1.0,1.0,218.0,3.0,8.0,11.0,2.015101e+09,3.0,53.0,98.7,4.5,37.1,31.0,9.0,464.0,0.0,1600.0,16.210740,4.0,0.114224,1.0,11.0,0.0,0.0,12.0,0.0,1.0,1.0,232.0,8.0,2.0,3.0,2.015101e+09,3.0,53.0,100.1,13.0,37.2,12.9,6.0,460.0,0.0,1600.0,15.984016,-8.0,0.115217,1.0,16.0,0.0,0.0,106.0,0.0,1.0,1.0,244.0,163.0,0.275862,0.048257,0.075,0.061194,0.044112,0.043171,0.05,0.99,2242.0,1.0,0.173913,0.0,0.1,0.230769,0.0,0.266667,1.5,425.0,0.534884,0.121032,0.388889,0.10241,0.115625,0.128676,0.112613,2.07,140.0,0.602151,0.107444,0.140351,0.090129,0.112559,0.081664,0.130146,2.94
1,201805050108,9.0,6.0,7.0,3.0,54.0,99.8,10.5,38.0,17.9,5.0,462.0,2018-11-03 13:55:00,0.0,0.0,1.0,0.0,1600.0,16.032064,-2.0,0.116883,1.0,11.0,0.0,0.0,14.0,1.0,1.0,1.0,3.0,8.0,11.0,2.015101e+09,3.0,53.0,98.7,4.5,37.1,31.0,9.0,464.0,0.0,1600.0,16.210740,4.0,0.114224,1.0,11.0,0.0,0.0,12.0,0.0,1.0,1.0,14.0,8.0,2.0,3.0,2.015101e+09,3.0,53.0,100.1,13.0,37.2,12.9,6.0,460.0,0.0,1600.0,15.984016,-8.0,0.115217,1.0,16.0,0.0,0.0,106.0,0.0,1.0,1.0,26.0,1.0,2.0,2.0,2.015101e+09,3.0,54.0,97.4,11.0,36.2,7.2,3.0,464.0,0.0,1600.0,16.427105,-4.0,0.116379,2.0,15.0,1.0,0.0,13.0,0.0,1.0,1.0,132.0,163.0,0.275862,0.048257,0.075,0.061194,0.044112,0.043171,0.05,0.99,2242.0,1.0,0.173913,0.0,0.1,0.230769,0.0,0.266667,1.5,425.0,0.534884,0.121032,0.388889,0.10241,0.115625,0.128676,0.112613,2.07,140.0,0.602151,0.107444,0.140351,0.090129,0.112559,0.081664,0.130146,2.94
2,201805040607,3.0,8.0,11.0,3.0,53.0,98.7,4.5,37.1,31.0,9.0,464.0,2018-10-20 13:35:00,0.0,0.0,1.0,0.0,1600.0,16.210740,4.0,0.114224,1.0,11.0,0.0,0.0,12.0,0.0,1.0,1.0,8.0,2.0,3.0,2.015101e+09,3.0,53.0,100.1,13.0,37.2,12.9,6.0,460.0,0.0,1600.0,15.984016,-8.0,0.115217,1.0,16.0,0.0,0.0,106.0,0.0,1.0,1.0,12.0,1.0,2.0,2.0,2

## target の作成

In [120]:
final_df = df

In [121]:
# 正解ラベル
final_df["is_tansyo"] = (final_df["rank"]<=1.1) * 1
final_df["is_hukusyo"] = (final_df["rank"]<=3.1) * 1

#（着順上位(1~3着)/中位(4~7着)/下位(8~)の予測）
final_df.loc[final_df["rank"]<=3.1, "rank_category"] = 0
final_df.loc[3.1<final_df["rank"], "rank_category"] = 1
final_df.loc[7.1<final_df["rank"], "rank_category"] = 2

final_df['is_tansyo'] = final_df['is_tansyo'].astype(int)
final_df['is_hukusyo'] = final_df['is_hukusyo'].astype(int)
final_df['rank_category'] = final_df['rank_category'].astype(int)

In [122]:
final_df['rank_category'].value_counts()

2    24839
1    12171
0     9130
Name: rank_category, dtype: int64

In [123]:
final_df.head(3)

,race_id,rank,frame_number,horse_number,age,burden_weight,goal_time,half_way_rank,last_time,odds,popular,horse_weight,date,is_down,is_senba,is_mesu,is_osu,distance,avg_velocity,horse_weight_dif,burden_weight_rate,ground_status,total_horse_number,weather_rain,weather_snow,pre_date_diff,is_rider_same,is_tamer_same,is_owner_same,rank_1,frame_number_1,horse_number_1,horse_id_1,age_1,burden_weight_1,goal_time_1,half_way_rank_1,last_time_1,odds_1,popular_1,horse_weight_1,is_down_1,distance_1,avg_velocity_1,horse_weight_dif_1,burden_weight_rate_1,ground_status_1,total_horse_number_1,weather_rain_1,weather_snow_1,pre_date_diff_1,is_rider_same_1,is_tamer_same_1,is_owner_same_1,interval_date_1,rank_2,frame_number_2,horse_number_2,horse_id_2,age_2,burden_weight_2,goal_time_2,half_way_rank_2,last_time_2,odds_2,popular_2,horse_weight_2,is_down_2,distance_2,avg_velocity_2,horse_weight_dif_2,burden_weight_rate_2,ground_status_2,total_horse_number_2,weather_rain_2,weather_snow_2,pre_date_diff_2,is_rider_same_2,is_tamer_same_2,is_owner_same_2,interval_date_2,rank_3,frame_number_3,horse_number_3,horse_id_3,age_3,burden_weight_3,goal_time_3,half_way_rank_3,last_time_3,odds_3,popular_3,horse_weight_3,is_down_3,distance_3,avg_velocity_3,horse_weight_dif_3,burden_weight_rate_3,ground_status_3,total_horse_number_3,weather_rain_3,weather_snow_3,pre_date_diff_3,is_rider_same_3,is_tamer_same_3,is_owner_same_3,interval_date_3,rank_f,win_horse_rate,win_times_rate,win_jusyo_rate,win_special_rate,win_plane_rate,win_turf_rate,win_dirt_rate,EI,rank_mf,win_horse_rate_mf,win_times_rate_mf,win_jusyo_rate_mf,win_special_rate_mf,win_plane_rate_mf,win_turf_rate_mf,win_dirt_rate_mf,EI_mf,rank_mmf,win_horse_rate_mmf,win_times_rate_mmf,win_jusyo_rate_mmf,win_special_rate_mmf,win_plane_rate_mmf,win_turf_rate_mmf,win_dirt_rate_mmf,EI_mmf,rank_fmf,win_horse_rate_fmf,win_times_rate_fmf,win_jusyo_rate_fmf,win_special_rate_fmf,win_plane_rate_fmf,win_turf_rate_fmf,win_dirt_rate_fmf,EI_fmf,is_tansyo,is_hukusyo,rank_category
0,201905030412,15.0,4.0,8.0,4.0,55.0,98.1,11.5,37.2,37.2,10.0,468.0,2019-06-09 16:25:00,0.0,0.0,1.0,0.0,1600.0,16.309888,6.0,0.117521,3.0,16.0,2.0,0.0,218.0,0.0,1.0,1.0,9.0,6.0,7.0,2.015101e+09,3.0,54.0,99.8,10.5,38.0,17.9,5.0,462.0,0.0,1600.0,16.032064,-2.0,0.116883,1.0,11.0,0.0,0.0,14.0,1.0,1.0,1.0,218.0,3.0,8.0,11.0,2.015101e+09,3.0,53.0,98.7,4.5,37.1,31.0,9.0,464.0,0.0,1600.0,16.210740,4.0,0.114224,1.0,11.0,0.0,0.0,12.0,0.0,1.0,1.0,232.0,8.0,2.0,3.0,2.015101e+09,3.0,53.0,100.1,13.0,37.2,12.9,6.0,460.0,0.0,1600.0,15.984016,-8.0,0.115217,1.0,16.0,0.0,0.0,106.0,0.0,1.0,1.0,244.0,163.0,0.275862,0.048257,0.075,0.061194,0.044112,0.043171,0.05,0.99,2242.0,1.0,0.173913,0.0,0.1,0.230769,0.0,0.266667,1.5,425.0,0.534884,0.121032,0.388889,0.10241,0.115625,0.128676,0.112613,2.07,140.0,0.602151,0.107444,0.140351,0.090129,0.112559,0.081664,0.130146,2.94,0,0,2
1,201805050108,9.0,6.0,7.0,3.0,54.0,99.8,10.5,38.0,17.9,5.0,462.0,2018-11-03 13:55:00,0.0,0.0,1.0,0.0,1600.0,16.032064,-2.0,0.116883,1.0,11.0,0.0,0.0,14.0,1.0,1.0,1.0,3.0,8.0,11.0,2.015101e+09,3.0,53.0,98.7,4.5,37.1,31.0,9.0,464.0,0.0,1600.0,16.210740,4.0,0.114224,1.0,11.0,0.0,0.0,12.0,0.0,1.0,1.0,14.0,8.0,2.0,3.0,2.015101e+09,3.0,53.0,100.1,13.0,37.2,12.9,6.0,460.0,0.0,1600.0,15.984016,-8.0,0.115217,1.0,16.0,0.0,0.0,106.0,0.0,1.0,1.0,26.0,1.0,2.0,2.0,2.015101e+09,3.0,54.0,97.4,11.0,36.2,7.2,3.0,464.0,0.0,1600.0,16.427105,-4.0,0.116379,2.0,15.0,1.0,0.0,13.0,0.0,1.0,1.0,132.0,163.0,0.275862,0.048257,0.075,0.061194,0.044112,0.043171,0.05,0.99,2242.0,1.0,0.173913,0.0,0.1,0.230769,0.0,0.266667,1.5,425.0,0.534884,0.121032,0.388889,0.10241,0.115625,0.128676,0.112613,2.07,140.0,0.602151,0.107444,0.140351,0.090129,0.112559,0.081664,0.130146,2.94,0,0,2
2,201805040607,3.0,8.0,11.0,3.0,53.0,98.7,4.5,37.1,31.0,9.0,464.0,2018-10-20 13:35:00,0.0,0.0,1.0,0.0,1600.0,16.210740,4.0,0.114224,1.0,11.0,0.0,0.0,12.0,0.0,1.0,1.0,8.0,2.0,3.0,2.015101e+09,3.0,53.0,100.1,13.0,37.2,12.9,6.0,460.0,0.0,1600.0,15.984016,-8.0,0.115217,1.0,16

In [124]:
"""
# もともとのdate情報, race_idはあとで必要なので保持
# 予想したいときのレース情報は
- frame_number
- horse_number
- age
- burden_weight
- horse_weight
- sex['is_senba','is_mesu','is_osu']
- distance
- horse_weight_dif
- burden_weight_rate
- ground_status
- total_horse_number
- is_obstacle
- weather_rain, weather_snow
- pre_date_diff
- is_rider_same_1	is_tamer_same_1	is_owner_same_1	interval_date_1

# あるとまずい情報は
- rank
- goal_time
- half_way_rank
- last_time
- odds
- popular
- is_down
- avg_velocity
- is_down

rank は正解ラベルを付けたあとに落とす

"""
final_df = final_df.reset_index(drop=True).sort_values("date")
final_df = final_df.drop(['rank', 'goal_time', 'half_way_rank','last_time', 'odds', 'popular', 'avg_velocity', 'is_down'], axis=1)
final_df = final_df.drop(['horse_id_1','horse_number_1','odds_1','popular_1'], axis=1)
final_df = final_df.drop(['horse_id_2','horse_number_2','odds_2','popular_2'], axis=1)
final_df = final_df.drop(['horse_id_3','horse_number_3','odds_3','popular_3'], axis=1)




In [88]:
'''
# 1つ前から先のレースデータがnullの馬がいるようなレースだったら削除する
print("null:{} shape:{}".format(final_df["rank_1"].isnull().sum(),final_df.shape))
invalid_race = final_df.loc[final_df["rank_1"].isnull(), "race_id"].unique()
print(invalid_race)
deleted_df = final_df[~final_df["race_id"].isin(invalid_race)].copy()
print("null:{} shape:{}".format(deleted_df["rank_1"].isnull().sum(),deleted_df.shape))

'''


'\n# 1つ前から先のレースデータがnullの馬がいるようなレースだったら削除する\nprint("null:{} shape:{}".format(final_df["rank_1"].isnull().sum(),final_df.shape))\ninvalid_race = final_df.loc[final_df["rank_1"].isnull(), "race_id"].unique()\nprint(invalid_race)\ndeleted_df = final_df[~final_df["race_id"].isin(invalid_race)].copy()\nprint("null:{} shape:{}".format(deleted_df["rank_1"].isnull().sum(),deleted_df.shape))\n\n'

In [125]:
print(final_df.isnull().sum())

race_id                     0
frame_number                0
horse_number                0
age                         0
burden_weight               0
horse_weight                0
date                        0
is_senba                    0
is_mesu                     0
is_osu                      0
distance                    0
horse_weight_dif            0
burden_weight_rate          0
ground_status               0
total_horse_number          0
weather_rain                0
weather_snow                0
pre_date_diff           18500
is_rider_same               0
is_tamer_same               0
is_owner_same               0
rank_1                  18500
frame_number_1          18500
age_1                   18500
burden_weight_1         18500
goal_time_1             18500
half_way_rank_1         18500
last_time_1             18500
horse_weight_1          18500
is_down_1               18500
distance_1              18500
avg_velocity_1          18500
horse_weight_dif_1      18500
burden_wei

In [113]:
final_df.sort_values("date").head()

,race_id,frame_number,horse_number,age,burden_weight,horse_weight,date,is_senba,is_mesu,is_osu,distance,horse_weight_dif,burden_weight_rate,ground_status,total_horse_number,weather_rain,weather_snow,pre_date_diff,is_rider_same,is_tamer_same,is_owner_same,rank_1,frame_number_1,horse_number_1,horse_id_1,age_1,burden_weight_1,goal_time_1,half_way_rank_1,last_time_1,odds_1,popular_1,horse_weight_1,is_down_1,distance_1,avg_velocity_1,horse_weight_dif_1,burden_weight_rate_1,ground_status_1,total_horse_number_1,weather_rain_1,weather_snow_1,pre_date_diff_1,is_rider_same_1,is_tamer_same_1,is_owner_same_1,interval_date_1,rank_2,frame_number_2,horse_number_2,horse_id_2,age_2,burden_weight_2,goal_time_2,half_way_rank_2,last_time_2,odds_2,popular_2,horse_weight_2,is_down_2,distance_2,avg_velocity_2,horse_weight_dif_2,burden_weight_rate_2,ground_status_2,total_horse_number_2,weather_rain_2,weather_snow_2,pre_date_diff_2,is_rider_same_2,is_tamer_same_2,is_owner_same_2,interval_date_2,rank_3,frame_number_3,horse_number_3,horse_id_3,age_3,burden_weight_3,goal_time_3,half_way_rank_3,last_time_3,odds_3,popular_3,horse_weight_3,is_down_3,distance_3,avg_velocity_3,horse_weight_dif_3,burden_weight_rate_3,ground_status_3,total_horse_number_3,weather_rain_3,weather_snow_3,pre_date_diff_3,is_rider_same_3,is_tamer_same_3,is_owner_same_3,interval_date_3,rank_f,win_horse_rate,win_times_rate,win_jusyo_rate,win_special_rate,win_plane_rate,win_turf_rate,win_dirt_rate,EI,rank_mf,win_horse_rate_mf,win_times_rate_mf,win_jusyo_rate_mf,win_special_rate_mf,win_plane_rate_mf,win_turf_rate_mf,win_dirt_rate_mf,EI_mf,rank_mmf,win_horse_rate_mmf,win_times_rate_mmf,win_jusyo_rate_mmf,win_special_rate_mmf,win_plane_rate_mmf,win_turf_rate_mmf,win_dirt_rate_mmf,EI_mmf,rank_fmf,win_horse_rate_fmf,win_times_rate_fmf,win_jusyo_rate_fmf,win_special_rate_fmf,win_plane_rate_fmf,win_turf_rate_fmf,win_dirt_rate_fmf,EI_fmf,is_tansyo,is_hukusyo,rank_category
31999,200805010101,2.0,4.0,3.0,54.0,416.0,2008-02-02 10:00:00,0.0,1.0,0.0,1400.0,-8.0,0.129808,1.0,16.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.0,0.268657,0.067251,0.087209,0.075549,0.063660,0.077361,0.051650,0.83,12.0,0.514839,0.105442,0.091729,0.107819,0.105663,0.112040,0.086993,2.05,756.0,0.687500,0.193878,0.10000,0.142857,0.223881,0.238095,0.142857,2.73,2598.0,0.500000,0.176471,0.000000,0.000000,0.230769,0.142857,0.200000,0.42,0,0,2
15146,200805010101,5.0,10.0,3.0,54.0,486.0,2008-02-02 10:00:00,0.0,1.0,0.0,1400.0,-8.0,0.111111,1.0,16.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.0,0.312139,0.072889,0.088339,0.104167,0.060677,0.082045,0.044890,1.33,41.0,0.513089,0.106312,0.108392,0.114407,0.102441,0.116063,0.083879,1.76,6.0,0.668981,0.131692,0.08445,0.110578,0.143136,0.113770,0.143619,2.32,12.0,0.514839,0.105442,0.091729,0.107819,0.105663,0.112040,0.086993,2.05,0,0,2
44019,200805010101,6.0,11.0,3.0,54.0,472.0,2008-02-02 10:00:00,0.0,1.0,0.0,1400.0,12.0,0.114407,1.0,16.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,556.0,0.233766,0.043980,0.000000,0.040000,0.044888,0.041734,0.044160,0.41,133.0,0.514493,0.087540,0.080645,0.076125,0.088645,0.071880,0.093555,

In [126]:
import datetime as dt
# 2009年以降は検証用のデータにする
final_df =final_df.sort_values("date")
finel_df_train = final_df[final_df["date"]<dt.datetime(2019,1,1)]
finel_df_test = final_df[final_df["date"]>dt.datetime(2019,1,1)]

In [127]:
# データの保存
final_df.sort_values("date").to_csv("csv_merged/final_data.csv", index=False)
finel_df_train.sort_values("date").to_csv("csv_merged/finel_data_train.csv", index=False)
finel_df_test.sort_values("date").to_csv("csv_merged/finel_data_test.csv", index=False)